In [1]:
import sys
sys.path.append('../src/')

#import warnings
#warnings.filterwarnings('ignore')  # Suppress warnings for cleaner output

from pathlib import Path
import json
import os

from subject import Subject
from physio_recording import PhysioRecording
from session import Session
from helpers import *

In [2]:
coi_structure_pathname = Path("../data/coi_structure.json")

if not coi_structure_pathname.exists():
    raise FileNotFoundError("The COI structure file does not exist")

with open(coi_structure_pathname, "r") as f:
    coi_structure = json.load(f)

In [3]:
FAMILY_ID = 4
SEANCE_ID = 2
SESSION_ID = 0

session = Session(session_id = SESSION_ID, family_id = FAMILY_ID, seance_id = SEANCE_ID)

subjects: [Subject] = []

subjects.append(Subject(id = 0, role_id = 0, role_desc = "THERAPIST"))
subjects.append(Subject(id = 1, role_id = 1, role_desc = "MOTHER"))
subjects.append(Subject(id = 2, role_id = 2, role_desc = "CHILD"))

for subject in subjects:
    physio_recording = PhysioRecording(subject_id = subject.id, session_id = SESSION_ID)
    physio_filepath = extract_raw_pathname(coi_structure, FAMILY_ID = FAMILY_ID, SEANCE_ID = SEANCE_ID, ROLE_ID = subject.role_id)
    physio_recording.set_physio_filepath(physio_filepath = physio_filepath)
    session.add_physio_recording(physio_recording)

In [4]:
session.load_physio_recordings_data()


Loading raw data for session 0 and subject 0
Loaded resting state Temperature data from TEMP_rs with 239 samples at 4 Hz.
Loaded session Temperature data from TEMP_session with 16115 samples at 4 Hz.
Loaded resting state EDA data from EDA_rs with 239 samples at 4 Hz.
Loaded session EDA data from EDA_session with 16117 samples at 4 Hz.
Loaded resting state BVP data from BVP_rs with 3839 samples at 64 Hz.
Loaded session BVP data from BVP_session with 257917 samples at 64 Hz.

Loading raw data for session 0 and subject 1
Loaded resting state Temperature data from TEMP_rs with 0 samples at 4 Hz.
Loaded session Temperature data from TEMP_session with 16131 samples at 4 Hz.
Loaded resting state EDA data from EDA_rs with 239 samples at 4 Hz.
Loaded session EDA data from EDA_session with 15887 samples at 4 Hz.
Loaded resting state BVP data from BVP_rs with 3839 samples at 64 Hz.
Loaded session BVP data from BVP_session with 254244 samples at 64 Hz.

Loading raw data for session 0 and subject 